Imports

In [111]:
import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding

In [112]:
train_file = 'data/train.tsv'
validation_percent = 0.1
test_file = ''
using_word_pairs = False

In [113]:
print("Loading training data...")
labels = []
inputs = []
with open(train_file, encoding='utf-8') as data:
  reader = csv.reader(data, delimiter='\t')
  for row in reader:
    if len(row) == 2:
        labels.append(row[0])
        inputs.append(row[1])
print("Loaded {} documents".format(len(labels)))

Loading training data...
Loaded 78608 documents


In [114]:
data = {'labels': labels, 'inputs': inputs}
train_data = pd.DataFrame(data=data)

In [115]:
X_train, X_valid, y_train_str, y_valid_str = train_test_split(train_data['inputs'], train_data['labels'], test_size=0.2, random_state = 42)
y_train = [int(a) for a in y_train_str]
y_valid = [int(a) for a in y_valid_str]

In [116]:
V = 10000
tokenizer = Tokenizer(num_words=V)
tokenizer.fit_on_texts(train_data['inputs'])

In [117]:
max_length = max([len(document.split()) for document in train_data['inputs']])
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_valid_seq = tokenizer.texts_to_sequences(X_valid)
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_valid_padded = pad_sequences(X_valid_seq, maxlen=max_length, padding='post')

In [118]:
emb_dim = 100
model = Sequential()
model.add(Embedding(input_dim=V, output_dim=emb_dim, input_length=max_length))
model.add(LSTM(units=100,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 2308, 100)         1000000   
_________________________________________________________________
lstm_7 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 101       
Total params: 1,080,501
Trainable params: 1,080,501
Non-trainable params: 0
_________________________________________________________________


In [119]:
model.fit(X_train_padded, np.asarray(y_train), batch_size=128, epochs=10, validation_data=(X_valid_padded, np.asarray(y_valid)),verbose=1)

ModuleNotFoundError: No module named 'keras'